# Multi-Regime Distribution Validation

## Objective
Test whether the diffusion model can learn **different probability distributions** for **different input conditions**.

## Experiment Design

We create 3 distinct market regimes, each with a unique input pattern and target distribution:

| Regime | Input Pattern | Target Distribution |
|--------|---------------|--------------------|
| **Profit Mode** | Upward trend + low volatility | 80% profit (~+5%), 20% loss |
| **Loss Mode** | Downward trend + high volatility | 80% loss (~-3%), 20% profit |
| **Random Mode** | Flat/consolidation | Uniform, std ~3% |

## Success Criteria
- Model outputs different distributions for different input conditions
- Profit mode → right-skewed distribution (mostly positive)
- Loss mode → left-skewed distribution (mostly negative)
- Random mode → symmetric distribution around 0

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, Tuple, List

from src.models.model import StockDiffusionModel, ModelConfig

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Device setup
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

## 1. Define Market Regime Configurations

In [ ]:
# Experiment parameters
SEQ_LENGTH = 120
N_FEATURES = 4
FEATURE_NAMES = ['log_return', 'volatility', 'spread', 'volume_zscore']

# Regime configurations
REGIME_CONFIGS = {
    'profit': {
        'name': 'Profit Mode',
        'description': '80% win rate, avg +5% return',
        'win_rate': 0.80,
        'win_mean': 0.05,
        'win_std': 0.015,
        'lose_mean': -0.02,
        'lose_std': 0.01,
        'color': 'green'
    },
    'loss': {
        'name': 'Loss Mode', 
        'description': '80% lose rate, avg -3% return',
        'win_rate': 0.20,  # 20% win = 80% lose
        'win_mean': 0.02,
        'win_std': 0.01,
        'lose_mean': -0.03,
        'lose_std': 0.012,
        'color': 'red'
    },
    'random': {
        'name': 'Random Mode',
        'description': 'No pattern, std ~3%',
        'distribution': 'uniform',
        'std': 0.03,
        'color': 'gray'
    }
}

print("Market Regime Configurations:")
print("=" * 50)
for key, cfg in REGIME_CONFIGS.items():
    print(f"\n{cfg['name']}:")
    print(f"  {cfg['description']}")

## 2. Create Regime-Specific Datasets

In [ ]:
class RegimeDataset(Dataset):
    """
    Dataset for a specific market regime.
    
    Each regime has:
    - A unique input condition pattern
    - A specific target distribution
    """
    
    def __init__(
        self,
        regime: str,
        n_samples: int = 1000,
        seq_length: int = 120,
        n_features: int = 4
    ):
        self.regime = regime
        self.n_samples = n_samples
        self.seq_length = seq_length
        self.n_features = n_features
        self.config = REGIME_CONFIGS[regime]
        
        # Create condition pattern for this regime
        self.condition = self._create_condition()
        
        # Generate targets according to regime distribution
        self.targets = self._generate_targets()
        
    def _create_condition(self) -> np.ndarray:
        """Create a unique condition pattern for this regime."""
        t = np.linspace(0, 4 * np.pi, self.seq_length)
        features = np.zeros((self.seq_length, self.n_features), dtype=np.float32)
        
        if self.regime == 'profit':
            # Upward trend pattern with low volatility
            features[:, 0] = 0.002 + 0.001 * np.sin(t)  # Positive drift in returns
            features[:, 1] = 0.003 + 0.001 * np.cos(t)  # Low, stable volatility
            features[:, 2] = 0.0005 * np.ones(self.seq_length)  # Tight spread
            features[:, 3] = 0.5 + 0.3 * np.sin(t / 2)  # Moderate positive volume
            
        elif self.regime == 'loss':
            # Downward trend pattern with high volatility
            features[:, 0] = -0.002 + 0.002 * np.sin(2 * t)  # Negative drift
            features[:, 1] = 0.008 + 0.003 * np.abs(np.sin(t))  # High volatility
            features[:, 2] = 0.002 + 0.001 * np.sin(t)  # Wide spread
            features[:, 3] = -0.3 + 0.5 * np.cos(t)  # Erratic volume
            
        elif self.regime == 'random':
            # Flat/consolidation pattern
            features[:, 0] = 0.0001 * np.sin(3 * t)  # Near-zero returns
            features[:, 1] = 0.005 * np.ones(self.seq_length)  # Medium volatility
            features[:, 2] = 0.001 * np.ones(self.seq_length)  # Normal spread
            features[:, 3] = 0.2 * np.sin(t / 3)  # Low volume variation
            
        return features
    
    def _generate_targets(self) -> np.ndarray:
        """Generate targets according to regime distribution."""
        targets = np.zeros(self.n_samples, dtype=np.float32)
        
        if self.regime in ['profit', 'loss']:
            # Asymmetric distribution: win_rate% positive, (1-win_rate)% negative
            cfg = self.config
            n_wins = int(self.n_samples * cfg['win_rate'])
            n_losses = self.n_samples - n_wins
            
            # Generate winning samples (positive returns)
            wins = np.random.normal(cfg['win_mean'], cfg['win_std'], n_wins)
            wins = np.clip(wins, 0.001, 0.15)  # Ensure positive
            
            # Generate losing samples (negative returns)
            losses = np.random.normal(cfg['lose_mean'], cfg['lose_std'], n_losses)
            losses = np.clip(losses, -0.15, -0.001)  # Ensure negative
            
            targets[:n_wins] = wins
            targets[n_wins:] = losses
            np.random.shuffle(targets)
            
        elif self.regime == 'random':
            # Uniform distribution centered at 0
            std = self.config['std']
            # Uniform in range [-sqrt(3)*std, sqrt(3)*std] gives desired std
            half_range = np.sqrt(3) * std
            targets = np.random.uniform(-half_range, half_range, self.n_samples).astype(np.float32)
            
        return targets
    
    def __len__(self) -> int:
        return self.n_samples
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        return {
            'input': torch.from_numpy(self.condition.copy()),
            'target': torch.tensor([self.targets[idx]], dtype=torch.float32),
            'regime': self.regime
        }
    
    def get_condition_tensor(self) -> torch.Tensor:
        """Get the condition as a tensor for inference."""
        return torch.from_numpy(self.condition.copy())
    
    def get_stats(self) -> Dict:
        """Get statistics of this regime's targets."""
        positive = (self.targets > 0).sum()
        negative = (self.targets < 0).sum()
        return {
            'regime': self.regime,
            'n_samples': self.n_samples,
            'mean': self.targets.mean(),
            'std': self.targets.std(),
            'min': self.targets.min(),
            'max': self.targets.max(),
            'pct_positive': positive / self.n_samples * 100,
            'pct_negative': negative / self.n_samples * 100
        }

In [ ]:
# Create datasets for each regime
N_SAMPLES_PER_REGIME = 2000

datasets = {}
for regime in ['profit', 'loss', 'random']:
    datasets[regime] = RegimeDataset(
        regime=regime,
        n_samples=N_SAMPLES_PER_REGIME,
        seq_length=SEQ_LENGTH,
        n_features=N_FEATURES
    )

# Print statistics
print("Dataset Statistics:")
print("=" * 70)
print(f"{'Regime':<12} {'Samples':<10} {'Mean':>10} {'Std':>10} {'%Positive':>12} {'%Negative':>12}")
print("-" * 70)
for regime, ds in datasets.items():
    stats = ds.get_stats()
    print(f"{stats['regime']:<12} {stats['n_samples']:<10} {stats['mean']:>10.4f} {stats['std']:>10.4f} {stats['pct_positive']:>11.1f}% {stats['pct_negative']:>11.1f}%")

In [ ]:
# Visualize conditions and target distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for idx, (regime, ds) in enumerate(datasets.items()):
    cfg = REGIME_CONFIGS[regime]
    
    # Top row: Input conditions
    ax = axes[0, idx]
    for i in range(N_FEATURES):
        ax.plot(ds.condition[:, i], label=FEATURE_NAMES[i], alpha=0.8)
    ax.set_title(f"{cfg['name']}\nInput Condition")
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Feature Value')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Bottom row: Target distributions
    ax = axes[1, idx]
    ax.hist(ds.targets * 100, bins=50, color=cfg['color'], alpha=0.7, edgecolor='black')
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
    ax.axvline(x=ds.targets.mean() * 100, color='blue', linestyle='-', linewidth=2, label=f'Mean: {ds.targets.mean()*100:.2f}%')
    ax.set_title(f"{cfg['name']}\nTarget Distribution")
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Count')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Create Combined Dataset and Model

In [ ]:
# Combine all regime datasets for training
combined_dataset = ConcatDataset(list(datasets.values()))
print(f"Combined dataset size: {len(combined_dataset)} samples")

# Create model
model_config = ModelConfig(
    encoder_type="transformer",
    input_features=N_FEATURES,
    seq_length=SEQ_LENGTH,
    encoder_dim=64,       # Larger for multi-regime
    encoder_layers=2,
    encoder_heads=4,
    diffusion_steps=500,
    noise_schedule="cosine",
    prediction_type="epsilon",
    denoiser_type="mlp",
    denoising_hidden_dim=128,
    denoising_blocks=4,
    time_embedding_dim=64,
    dropout=0.1
)

model = StockDiffusionModel(model_config)
model = model.to(device)

print(f"Model parameters: {model.get_num_params():,}")

## 4. Train the Model

In [ ]:
def train_model(
    model: nn.Module,
    dataset: Dataset,
    epochs: int = 150,
    batch_size: int = 64,
    learning_rate: float = 5e-4,
    device: torch.device = device
) -> List[float]:
    """Train the diffusion model."""
    
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True,
        drop_last=True
    )
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs, eta_min=learning_rate / 20
    )
    
    losses = []
    model.train()
    
    pbar = tqdm(range(epochs), desc="Training")
    for epoch in pbar:
        epoch_loss = 0.0
        n_batches = 0
        
        for batch in dataloader:
            x_seq = batch['input'].to(device)
            target = batch['target'].to(device)
            
            optimizer.zero_grad()
            outputs = model(x_seq, target)
            loss = outputs['loss']
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
            n_batches += 1
        
        scheduler.step()
        avg_loss = epoch_loss / n_batches
        losses.append(avg_loss)
        
        pbar.set_postfix({'loss': f'{avg_loss:.6f}', 'lr': f'{scheduler.get_last_lr()[0]:.2e}'})
    
    return losses

In [ ]:
# Train the model
EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 5e-4

print(f"Training configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Total samples: {len(combined_dataset)}")
print()

losses = train_model(
    model, 
    combined_dataset, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE
)

# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print(f"\nFinal loss: {losses[-1]:.6f}")

## 5. Sample from Each Regime

Now we test if the model learned to output different distributions for different input conditions.

In [ ]:
@torch.no_grad()
def sample_regime(
    model: nn.Module,
    condition: torch.Tensor,
    n_samples: int = 500,
    use_ddim: bool = True,
    ddim_steps: int = 50,
    device: torch.device = device
) -> np.ndarray:
    """Sample predictions for a given condition."""
    model.eval()
    
    condition = condition.unsqueeze(0).to(device)
    condition = condition.expand(n_samples, -1, -1)
    
    samples = model.sample(
        condition,
        num_samples=1,
        use_ddim=use_ddim,
        ddim_steps=ddim_steps
    )
    
    return samples.cpu().numpy().flatten()

In [ ]:
# Sample from each regime
N_INFERENCE_SAMPLES = 500

print(f"Sampling {N_INFERENCE_SAMPLES} predictions for each regime...")
print()

predictions = {}
for regime, ds in datasets.items():
    condition = ds.get_condition_tensor()
    preds = sample_regime(
        model,
        condition,
        n_samples=N_INFERENCE_SAMPLES,
        use_ddim=True,
        ddim_steps=50
    )
    predictions[regime] = preds
    
    cfg = REGIME_CONFIGS[regime]
    print(f"{cfg['name']}:")
    print(f"  Mean: {preds.mean()*100:+.2f}%")
    print(f"  Std:  {preds.std()*100:.2f}%")
    print(f"  Positive: {(preds > 0).sum()/len(preds)*100:.1f}%")
    print()

## 6. Visualize and Compare Results

In [ ]:
# Compare ground truth vs model predictions for each regime
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

for idx, (regime, ds) in enumerate(datasets.items()):
    cfg = REGIME_CONFIGS[regime]
    preds = predictions[regime]
    
    # Top row: Ground truth distributions
    ax = axes[0, idx]
    ax.hist(ds.targets * 100, bins=50, color=cfg['color'], alpha=0.7, 
            edgecolor='black', label='Ground Truth', density=True)
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
    ax.axvline(x=ds.targets.mean() * 100, color='blue', linestyle='-', 
               linewidth=2, label=f'Mean: {ds.targets.mean()*100:+.2f}%')
    ax.set_title(f"{cfg['name']}\nGround Truth Distribution", fontsize=12)
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-12, 12)
    
    # Bottom row: Model predictions
    ax = axes[1, idx]
    ax.hist(preds * 100, bins=50, color=cfg['color'], alpha=0.7, 
            edgecolor='black', label='Model Output', density=True)
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
    ax.axvline(x=preds.mean() * 100, color='blue', linestyle='-', 
               linewidth=2, label=f'Mean: {preds.mean()*100:+.2f}%')
    ax.axvline(x=ds.targets.mean() * 100, color='red', linestyle=':', 
               linewidth=2, label=f'Expected: {ds.targets.mean()*100:+.2f}%')
    ax.set_title(f"{cfg['name']}\nModel Predictions (n={N_INFERENCE_SAMPLES})", fontsize=12)
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-12, 12)

plt.tight_layout()
plt.savefig(project_root / 'outputs' / 'regime_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/regime_comparison.png")

In [ ]:
# Overlay all predictions on single plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Ground truth overlay
ax = axes[0]
for regime, ds in datasets.items():
    cfg = REGIME_CONFIGS[regime]
    ax.hist(ds.targets * 100, bins=50, color=cfg['color'], alpha=0.5, 
            label=f"{cfg['name']} (μ={ds.targets.mean()*100:+.1f}%)", density=True)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
ax.set_title('Ground Truth Distributions (All Regimes)', fontsize=12)
ax.set_xlabel('Return (%)')
ax.set_ylabel('Density')
ax.legend()
ax.grid(True, alpha=0.3)

# Right: Model predictions overlay
ax = axes[1]
for regime, preds in predictions.items():
    cfg = REGIME_CONFIGS[regime]
    ax.hist(preds * 100, bins=50, color=cfg['color'], alpha=0.5, 
            label=f"{cfg['name']} (μ={preds.mean()*100:+.1f}%)", density=True)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
ax.set_title('Model Predictions (All Regimes)', fontsize=12)
ax.set_xlabel('Return (%)')
ax.set_ylabel('Density')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Quantitative Analysis

In [ ]:
def analyze_regime_results(ground_truth: np.ndarray, predictions: np.ndarray, regime: str) -> Dict:
    """Analyze how well the model captures the regime distribution."""
    
    gt_positive = (ground_truth > 0).sum() / len(ground_truth) * 100
    pred_positive = (predictions > 0).sum() / len(predictions) * 100
    
    results = {
        'regime': regime,
        # Mean comparison
        'gt_mean': ground_truth.mean(),
        'pred_mean': predictions.mean(),
        'mean_error': abs(predictions.mean() - ground_truth.mean()),
        # Std comparison
        'gt_std': ground_truth.std(),
        'pred_std': predictions.std(),
        'std_ratio': predictions.std() / ground_truth.std(),
        # Positive ratio comparison
        'gt_pct_positive': gt_positive,
        'pred_pct_positive': pred_positive,
        'positive_error': abs(pred_positive - gt_positive),
        # Distribution shape
        'gt_skew': float(np.mean((ground_truth - ground_truth.mean())**3) / ground_truth.std()**3),
        'pred_skew': float(np.mean((predictions - predictions.mean())**3) / predictions.std()**3) if predictions.std() > 0 else 0,
    }
    
    return results

In [ ]:
# Analyze each regime
print("=" * 80)
print("QUANTITATIVE ANALYSIS")
print("=" * 80)

all_results = []
for regime, ds in datasets.items():
    results = analyze_regime_results(ds.targets, predictions[regime], regime)
    all_results.append(results)
    
    cfg = REGIME_CONFIGS[regime]
    print(f"\n{cfg['name']} ({cfg['description']})")
    print("-" * 60)
    print(f"  {'Metric':<25} {'Ground Truth':>15} {'Prediction':>15} {'Error':>10}")
    print(f"  {'-'*65}")
    print(f"  {'Mean Return':<25} {results['gt_mean']*100:>14.2f}% {results['pred_mean']*100:>14.2f}% {results['mean_error']*100:>9.2f}%")
    print(f"  {'Std Dev':<25} {results['gt_std']*100:>14.2f}% {results['pred_std']*100:>14.2f}% {abs(results['gt_std']-results['pred_std'])*100:>9.2f}%")
    print(f"  {'% Positive':<25} {results['gt_pct_positive']:>14.1f}% {results['pred_pct_positive']:>14.1f}% {results['positive_error']:>9.1f}%")
    print(f"  {'Skewness':<25} {results['gt_skew']:>15.2f} {results['pred_skew']:>15.2f}")

In [ ]:
# Final verdict
print("\n" + "=" * 80)
print("FINAL VERDICT")
print("=" * 80)

# Success criteria:
# 1. Profit mode should have positive mean and >60% positive predictions
# 2. Loss mode should have negative mean and >60% negative predictions  
# 3. Random mode should have near-zero mean and ~50% positive predictions
# 4. Model should output different distributions for different conditions

profit_preds = predictions['profit']
loss_preds = predictions['loss']
random_preds = predictions['random']

# Check each criterion
profit_correct = profit_preds.mean() > 0.01 and (profit_preds > 0).mean() > 0.6
loss_correct = loss_preds.mean() < -0.01 and (loss_preds < 0).mean() > 0.6
random_correct = abs(random_preds.mean()) < 0.015 and 0.3 < (random_preds > 0).mean() < 0.7

# Check if distributions are different
means_different = (
    profit_preds.mean() - loss_preds.mean() > 0.02 and  # profit > loss
    abs(profit_preds.mean() - random_preds.mean()) > 0.01 and  # profit != random
    abs(loss_preds.mean() - random_preds.mean()) > 0.01  # loss != random
)

print(f"\nCriteria Check:")
print(f"  [{'✓' if profit_correct else '✗'}] Profit mode: positive bias (mean={profit_preds.mean()*100:+.2f}%, positive={(profit_preds>0).mean()*100:.1f}%)")
print(f"  [{'✓' if loss_correct else '✗'}] Loss mode: negative bias (mean={loss_preds.mean()*100:+.2f}%, negative={(loss_preds<0).mean()*100:.1f}%)")
print(f"  [{'✓' if random_correct else '✗'}] Random mode: neutral (mean={random_preds.mean()*100:+.2f}%, positive={(random_preds>0).mean()*100:.1f}%)")
print(f"  [{'✓' if means_different else '✗'}] Distributions are distinct")

all_pass = profit_correct and loss_correct and random_correct and means_different

print("\n" + "-" * 60)
if all_pass:
    print("\n🎉 [PASS] Model successfully learned conditional distributions!")
    print("\nThe model correctly outputs:")
    print("  • Positive-skewed distribution for profit conditions")
    print("  • Negative-skewed distribution for loss conditions")
    print("  • Symmetric distribution for random conditions")
    print("\nThis demonstrates the model can:")
    print("  1. Learn different distributions for different inputs")
    print("  2. Capture asymmetric probability distributions")
    print("  3. Distinguish between market regimes")
elif profit_correct or loss_correct or random_correct:
    print("\n⚠️  [PARTIAL] Model learned some but not all regimes correctly.")
    print("\nConsider:")
    print("  • Training for more epochs")
    print("  • Increasing model capacity")
    print("  • Adjusting learning rate")
else:
    print("\n❌ [FAIL] Model failed to learn conditional distributions.")
    print("\nThe model may be:")
    print("  • Collapsing to a single distribution")
    print("  • Not learning the condition-to-distribution mapping")
    print("  • Underfitting the data")

print("\n" + "=" * 80)

## 8. Interactive Sampling Tool

Use this section to sample from specific regimes and visualize the results.

In [ ]:
def sample_and_visualize(regime: str, n_samples: int = 500):
    """
    Sample from a specific regime and visualize the results.
    
    Args:
        regime: One of 'profit', 'loss', 'random'
        n_samples: Number of samples to generate
    """
    if regime not in datasets:
        print(f"Unknown regime: {regime}. Choose from: {list(datasets.keys())}")
        return
    
    ds = datasets[regime]
    cfg = REGIME_CONFIGS[regime]
    
    print(f"Sampling {n_samples} predictions for {cfg['name']}...")
    
    # Sample
    condition = ds.get_condition_tensor()
    preds = sample_regime(model, condition, n_samples=n_samples)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Distribution
    ax = axes[0]
    ax.hist(ds.targets * 100, bins=40, alpha=0.5, label='Ground Truth', density=True)
    ax.hist(preds * 100, bins=40, alpha=0.5, label='Model Predictions', density=True)
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
    ax.set_title(f"{cfg['name']} - Distribution Comparison")
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Scatter of predictions
    ax = axes[1]
    ax.scatter(range(len(preds)), preds * 100, alpha=0.5, s=10, c=cfg['color'])
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax.axhline(y=ds.targets.mean() * 100, color='red', linestyle='-', linewidth=2, 
               label=f'Expected Mean: {ds.targets.mean()*100:+.2f}%')
    ax.axhline(y=preds.mean() * 100, color='blue', linestyle='-', linewidth=2,
               label=f'Actual Mean: {preds.mean()*100:+.2f}%')
    ax.set_title(f"{cfg['name']} - Individual Predictions")
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Predicted Return (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"\nStatistics:")
    print(f"  Ground Truth - Mean: {ds.targets.mean()*100:+.2f}%, Std: {ds.targets.std()*100:.2f}%, Positive: {(ds.targets>0).mean()*100:.1f}%")
    print(f"  Predictions  - Mean: {preds.mean()*100:+.2f}%, Std: {preds.std()*100:.2f}%, Positive: {(preds>0).mean()*100:.1f}%")
    
    return preds

In [ ]:
# Example: Sample from profit mode
profit_samples = sample_and_visualize('profit', n_samples=500)

In [ ]:
# Example: Sample from loss mode
loss_samples = sample_and_visualize('loss', n_samples=500)

In [ ]:
# Example: Sample from random mode
random_samples = sample_and_visualize('random', n_samples=500)

## 9. Save Results

In [ ]:
import json

# Save experiment results
experiment_results = {
    'config': {
        'n_samples_per_regime': N_SAMPLES_PER_REGIME,
        'seq_length': SEQ_LENGTH,
        'n_features': N_FEATURES,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'inference_samples': N_INFERENCE_SAMPLES
    },
    'regime_configs': {
        k: {kk: vv for kk, vv in v.items() if kk != 'color'} 
        for k, v in REGIME_CONFIGS.items()
    },
    'results': {
        regime: {
            'gt_mean': float(datasets[regime].targets.mean()),
            'gt_std': float(datasets[regime].targets.std()),
            'gt_pct_positive': float((datasets[regime].targets > 0).mean() * 100),
            'pred_mean': float(predictions[regime].mean()),
            'pred_std': float(predictions[regime].std()),
            'pred_pct_positive': float((predictions[regime] > 0).mean() * 100)
        }
        for regime in ['profit', 'loss', 'random']
    },
    'final_loss': float(losses[-1]),
    'verdict': {
        'profit_correct': bool(profit_correct),
        'loss_correct': bool(loss_correct),
        'random_correct': bool(random_correct),
        'means_different': bool(means_different),
        'all_pass': bool(all_pass)
    }
}

output_path = project_root / 'outputs' / 'multimodal_regime_results.json'
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(experiment_results, f, indent=2)

print(f"Results saved to: {output_path}")
print("\nExperiment complete!")